In [1]:
from simulator import Simulator
from utils import load_csv
from market import Market

### Initialise

In [2]:
simulator = Simulator(load_csv('data.csv'), 100, 40)

### Execute

In [3]:
for day in range(0, 1):
    simulator.execute_week()

buyer_products:{'police', 'wood', 'constructions_repairs', 'eggs', 'metal', 'transportation', 'health', 'law', 'vegetables', 'maze', 'council', 'meat', 'fish', 'cloth', 'wheat', 'tourism', 'milk', 'rice', 'fruits', 'chicken'}
seller_products:{'police', 'wood', 'constructions_repairs', 'eggs', 'metal', 'transportation', 'health', 'law', 'vegetables', 'maze', 'council', 'meat', 'fish', 'cloth', 'wheat', 'tourism', 'milk', 'rice', 'fruits', 'chicken'}
--------------------------------------------------------------------------------


In [5]:
# Scratchpad
Market.print_transactions(simulator.weeks[0]['transactions'])

1470 transactions
╒══════════════════════════╤═══════════════════════╤═════════════════╤══════════════════════════╤═══════════════════════╤═════════════════════════════╤══════════════╤═══════════════════╕
│ buyer                    │ product               │   product_units │ seller                   │ product_to_exchange   │   product_to_exchange_units │ trade_type   │   hours_exchanged │
╞══════════════════════════╪═══════════════════════╪═════════════════╪══════════════════════════╪═══════════════════════╪═════════════════════════════╪══════════════╪═══════════════════╡
│ 73:wood                  │ fruits                │            2    │ 32:fruits                │ wood                  │                        1.6  │ direct       │               4   │
├──────────────────────────┼───────────────────────┼─────────────────┼──────────────────────────┼───────────────────────┼─────────────────────────────┼──────────────┼───────────────────┤
│ 86:tourism               │ wood              

In [6]:
Simulator.print_village(simulator.weeks[0]['village'])

╒══════╤═══════════════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤══════════════╤═════════╤══════════════╤══════════════╤══════════════════╤═════════════╤═════════════╤═════════════════════════╤══════════════╤═════════════╤══════════════╕
│   id │ product               │        wheat │         rice │         maze │   vegetables │       fruits │         fish │      chicken │         meat │         eggs │         milk │   metal │         wood │        cloth │   transportation │     tourism │      health │   constructions_repairs │      council │         law │       police │
╞══════╪═══════════════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════╪══════════════╪══════════════╪══════════════════╪═════════════╪═════════════╪═════════════════════════╪══════════════╪══════════

In [ ]:
#x = 4
#print(f"{x} wheat = {x * exchange_rate['wheat']['vegetables']} vegetables")
#print(round(0.0001, 4) > 0)
print(4 in range(0, 5))

True
